# Day 5

>You're starting to sweat as the ship makes its way toward Mercury. The Elves suggest that you get the air conditioner working by upgrading your ship computer to support the Thermal Environment Supervision Terminal.


# Part 1

>The Thermal Environment Supervision Terminal (TEST) starts by running a diagnostic program (your puzzle input). The TEST diagnostic program will run on your existing Intcode computer after a few modifications:

>First, you'll need to add two new instructions:

>- Opcode 3 takes a single integer as input and saves it to the address given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
>- Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.

>Second, you'll need to add support for parameter modes:

>- Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory.
>- Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50

>**Parameters that an instruction writes to will never be in immediate mode.**

>The TEST diagnostic program will start by requesting from the user the ID of the system to test by running an input instruction - provide it 1, the ID for the ship's air conditioner unit.

>It will then perform a series of diagnostic tests confirming that various parts of the Intcode computer, like parameter modes, function correctly. For each test, it will run an output instruction indicating how far the result of the test was from the expected value, where 0 means the test was successful. Non-zero outputs mean that a function is not working correctly; check the instructions that were run before the output instruction to see which one failed.

What this means is that when we are running the intcode each time that it outputs a non-zero value, we need to halt the program. Otherwise, if it outputs zero (the end of the diagnostic test) then we can continue running the program until...

>Finally, the program will output a diagnostic code and immediately halt. This final output isn't an error; an output followed immediately by a halt means the program finished. If all outputs were zero except the diagnostic code, the diagnostic program ran successfully.

In [1]:
import math
from typing import Tuple, List, Union

In [2]:
def parse_opcode(opcode: int) -> Tuple[int]:
    opcode_param_list = [int(num) for num in str(opcode).zfill(5)]
    # Don't need the leading zero from the opcode
    return tuple(opcode_param_list[:3] + opcode_param_list[4:])

In [3]:
assert parse_opcode(3) == (0, 0, 0, 3)
assert parse_opcode(104) == (0, 0, 1, 4)
assert parse_opcode(1102) == (0, 1, 1, 2)

In [4]:
def get_output_positions(
    opcode_idx: int, first_param_mode: int, second_param_mode: int, intcode: List[int]
) -> Tuple[int]:
    
    first_output_position = opcode_idx + 1 if first_param_mode == 1 else intcode[opcode_idx + 1]
    
    try:
        second_output_position = opcode_idx + 2 if second_param_mode == 1 else intcode[opcode_idx + 2]
    except IndexError:
        return (first_output_position, 0)
    
    return (first_output_position, second_output_position)

In [5]:
 assert get_output_positions(
     opcode_idx=4, first_param_mode=0, second_param_mode=0, intcode=[1100,1,238,225,4,-1,0]
 ) == (-1, 0)
assert get_output_positions(
 opcode_idx=4, first_param_mode=1, second_param_mode=1, intcode=[1100,1,238,225,4,-1,0]
) == (5, 6)
assert get_output_positions(
    opcode_idx=4, first_param_mode=1, second_param_mode=0, intcode=[1100,1,238,225,104,0]
) == (5, 0)

In [6]:
def run_opcode(
    opcode: int, opcode_idx: int, intcode: List[int], opcode_input: int = None
) -> Union[List[int], int]:
    """Given an optcode and it's index in the intcode, perform the operations on the intcode
    
    Parameters
        opcode: A number no greater than 5 digits, ending in 01, 02, 03, 04
        opcode_idx: The opcode's current position in the intcode, or the address in the memory
        opcode_input: An input into the program. Only valid for opcode 3
        intcode: The intcode where the opcode is running
    
    Returns: 
       The modified intcode for opcodes 1, 2, 3, 7, and 8
       The output of the diagnostic test or diagnostic code for opcode 4
       The new opcode_idx for the intcode for opcode 5 and 6
    """
    # Third param mode is always in position mode
    _, second_param_mode, first_param_mode, opcode = parse_opcode(opcode)
    first_output_position, second_output_position = get_output_positions(
        opcode_idx, first_param_mode, second_param_mode, intcode
    )
    try:
        intcode_replace_idx = intcode[opcode_idx + 3]
    except IndexError:
        pass
    
    if opcode == 4:
        return intcode[first_output_position]
    elif opcode == 5 and intcode[first_output_position] != 0:
        return intcode[second_output_position]
    elif opcode == 6 and intcode[first_output_position] == 0:
        return intcode[second_output_position]
    
    
    if opcode == 1:
        intcode[intcode_replace_idx] = intcode[first_output_position] + intcode[second_output_position]
    elif opcode == 2:
        intcode[intcode_replace_idx] = intcode[first_output_position] * intcode[second_output_position]
    elif opcode == 3:
        intcode[first_output_position] = opcode_input
    elif opcode == 7:
        intcode[intcode_replace_idx] = 1 if intcode[first_output_position] < intcode[second_output_position] else 0
    elif opcode == 8:
        intcode[intcode_replace_idx] = 1 if intcode[first_output_position] == intcode[second_output_position] else 0
        
    return intcode 

In [7]:
# Tests for opcodes 7 and 8
assert run_opcode(opcode=8, opcode_idx=2, intcode=[3,9,8,9,10,9,4,9,99,7,8]) == [3,9,8,9,10,9,4,9,99,0,8]
assert run_opcode(opcode=7, opcode_idx=2, intcode=[3,9,7,9,10,9,4,9,99,2,8]) == [3,9,7,9,10,9,4,9,99,1,8]

assert run_opcode(opcode=1108, opcode_idx=2, intcode=[3,3,1108,8,8,3,4,3,99]) == [3,3,1108,1,8,3,4,3,99]
assert run_opcode(opcode=1107, opcode_idx=2, intcode=[3,3,1107,9,8,3,4,3,99]) == [3,3,1107,0,8,3,4,3,99]

In [8]:
# Tests for opcodes 5 and 6
test_intcode = [3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
assert run_opcode(opcode=1105, opcode_idx=2, intcode=test_intcode) == test_intcode
assert run_opcode(opcode=1105, opcode_idx=2, intcode=[3, 3, 1105, 100, 9, 1101, 0, 0, 12, 4, 12, 99, 1]) == 9

test_intcode = [3,12,6,12,15,1,13,14,13,4,13,99,100,0,1,9]
assert run_opcode(opcode=6, opcode_idx=2, intcode=test_intcode) == test_intcode
assert run_opcode(opcode=6, opcode_idx=2, intcode=[3,12,6,12,15,1,13,14,13,4,13,99,0,0,1,9])

In [9]:
assert run_opcode(opcode=104, opcode_idx=4, intcode=[1100,1,238,225,104,0]) == 0
assert run_opcode(opcode=4, opcode_idx=4, intcode=[1100,1,238,225,4,0]) == 1100
assert run_opcode(opcode=4, opcode_idx=4, intcode=[1100,1,238,225,4,-1,0]) == 0

assert run_opcode(opcode=3, opcode_idx=4, intcode=[1,1,1,1,3,2], opcode_input=1000) == [1,1,1000,1, 3, 2]
assert run_opcode(opcode=1003, opcode_idx=4, intcode=[1,1,1,1,3,2], opcode_input=1000) == [1,1,1000,1,3,2]

assert run_opcode(opcode=1101, opcode_idx=0, intcode=[1101,0,0,0,99]) == [0,0,0,0,99]
assert run_opcode(opcode=1102, opcode_idx=0, intcode=[1102,3,0,3,99]) == [1102,3,0,0,99]

assert run_opcode(opcode=101, opcode_idx=0, intcode=[1101,0,0,1,99]) == [1101,1101,0,1,99]
assert run_opcode(opcode=1002, opcode_idx=0, intcode=[1102,0,1,1,99]) == [1102,1102,1,1,99]

assert run_opcode(opcode=1101, opcode_idx=0, intcode=[1101,-1,-1,0,99]) == [-2,-1,-1,0,99]
assert run_opcode(opcode=1002, opcode_idx=0, intcode=[1102,-1,1,1,99]) == [1102,99,1,1,99]

In [10]:
# Previous tests should also still work
assert run_opcode(1, 0, [1,0,0,0,99]) == [2,0,0,0,99]
assert run_opcode(2, 0, [2,3,0,3,99]) == [2,3,0,6,99]
assert run_opcode(2, 0, [2,4,4,5,99,0]) == [2,4,4,5,99,9801]

In [11]:
def run_diagnostic_tests(intcode: List[int], diagnostic_input: int) -> int:
    """Keep running the optcodes in the intcode until 99 is reached"""
    intcode = intcode.copy()
    opcode_idx = 0
    # Assumed to be a 3 
    opcode = intcode[opcode_idx]
    # Run once with the given input, increment, then start loop
    intcode = run_opcode(opcode, opcode_idx, intcode, diagnostic_input)
    opcode_idx += 2
    
    while True:
        opcode = intcode[opcode_idx]
        opcode_num = int(str(opcode)[-1])
        # Look for exit first
        if opcode_num == 4:
            result = run_opcode(opcode, opcode_idx, intcode)
            end_of_program = intcode[opcode_idx + 2] == 99
            if result == 0 and not end_of_program:
                # Passed the test
                opcode_idx += 2
                continue
            elif result != 0 and not end_of_program:
                raise ValueError("Non-zero Test result before end of program")
            else:
                return result
        # Check opcodes that move the opcode_idx
        if opcode_num in [5, 6]:
            result = run_opcode(opcode, opcode_idx, intcode)
            opcode_idx = result if isinstance(result, int) else opcode_idx + 3
            continue
        # Run regular opcodes
        intcode = run_opcode(opcode, opcode_idx, intcode)
        opcode_idx += 4
        

In [12]:
assert run_diagnostic_tests(intcode=[3,4,1,0,1,5,4,0,99], diagnostic_input=2) == 3
assert run_diagnostic_tests(intcode=[3,4,2,0,1,5,4,5,99], diagnostic_input=2) == 6
assert run_diagnostic_tests(intcode=[3,3,1102,0,1,7,104,0,99], diagnostic_input=1) == 1
assert run_diagnostic_tests(intcode=[3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], diagnostic_input=0) == 0
assert run_diagnostic_tests(intcode=[3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9], diagnostic_input=10) == 1
assert run_diagnostic_tests(intcode=[3,3,1105,-1,9,1101,0,0,12,4,12,99,1], diagnostic_input=0) == 0
assert run_diagnostic_tests(intcode=[3,3,1105,-1,9,1101,0,0,12,4,12,99,1], diagnostic_input=7) == 1

# big_test_intcode = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]

# print(big_test_intcode, "\n")
# big_test_intcode1 = run_opcode(opcode=3, opcode_idx=0, intcode=big_test_intcode, opcode_input=7)
# print(big_test_intcode1, "\n")
# big_test_intcode2 = run_opcode(opcode=1108, opcode_idx=2, intcode=big_test_intcode1)
# print(big_test_intcode2, "\n")
# big_test_intcode3 =  run_opcode(opcode=1005, opcode_idx=6, intcode=big_test_intcode2)
# print(big_test_intcode3, "\n")
# big_test_intcode4 = run_opcode(opcode=107, opcode_idx=9, intcode=big_test_intcode3)
# print(big_test_intcode4, "\n")
# new_idx = run_opcode(opcode=1006, opcode_idx=13, intcode=big_test_intcode4)
# print(new_idx, "\n")
# run_opcode(opcode=104, opcode_idx=31, intcode=big_test_intcode4)

In [13]:
# Input given from puzzle
intcode = [3,225,1,225,6,6,1100,1,238,225,104,0,1101,33,37,225,101,6,218,224,1001,224,-82,224,4,224,102,8,223,223,101,7,224,224,1,223,224,223,1102,87,62,225,1102,75,65,224,1001,224,-4875,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,1102,49,27,225,1101,6,9,225,2,69,118,224,101,-300,224,224,4,224,102,8,223,223,101,6,224,224,1,224,223,223,1101,76,37,224,1001,224,-113,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,1101,47,50,225,102,43,165,224,1001,224,-473,224,4,224,102,8,223,223,1001,224,3,224,1,224,223,223,1002,39,86,224,101,-7482,224,224,4,224,102,8,223,223,1001,224,6,224,1,223,224,223,1102,11,82,225,1,213,65,224,1001,224,-102,224,4,224,1002,223,8,223,1001,224,6,224,1,224,223,223,1001,14,83,224,1001,224,-120,224,4,224,1002,223,8,223,101,1,224,224,1,223,224,223,1102,53,39,225,1101,65,76,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,1107,677,226,224,1002,223,2,223,1005,224,329,101,1,223,223,8,677,226,224,102,2,223,223,1006,224,344,1001,223,1,223,108,677,677,224,1002,223,2,223,1006,224,359,1001,223,1,223,1108,226,677,224,102,2,223,223,1006,224,374,1001,223,1,223,1008,677,226,224,102,2,223,223,1005,224,389,101,1,223,223,7,226,677,224,102,2,223,223,1005,224,404,1001,223,1,223,1007,677,677,224,1002,223,2,223,1006,224,419,101,1,223,223,107,677,226,224,102,2,223,223,1006,224,434,101,1,223,223,7,677,677,224,1002,223,2,223,1005,224,449,101,1,223,223,108,677,226,224,1002,223,2,223,1006,224,464,101,1,223,223,1008,226,226,224,1002,223,2,223,1006,224,479,101,1,223,223,107,677,677,224,1002,223,2,223,1006,224,494,1001,223,1,223,1108,677,226,224,102,2,223,223,1005,224,509,101,1,223,223,1007,226,677,224,102,2,223,223,1005,224,524,1001,223,1,223,1008,677,677,224,102,2,223,223,1005,224,539,1001,223,1,223,1107,677,677,224,1002,223,2,223,1006,224,554,1001,223,1,223,1007,226,226,224,1002,223,2,223,1005,224,569,1001,223,1,223,7,677,226,224,1002,223,2,223,1006,224,584,1001,223,1,223,108,226,226,224,102,2,223,223,1005,224,599,1001,223,1,223,8,677,677,224,102,2,223,223,1005,224,614,1001,223,1,223,1107,226,677,224,102,2,223,223,1005,224,629,1001,223,1,223,8,226,677,224,102,2,223,223,1006,224,644,1001,223,1,223,1108,226,226,224,1002,223,2,223,1006,224,659,101,1,223,223,107,226,226,224,1002,223,2,223,1006,224,674,1001,223,1,223,4,223,99,226]

In [14]:
run_diagnostic_tests(intcode, diagnostic_input=1)

16209841

# Part 2

So we got the air conditioner working :tada: but now it's putting the heat right back into the spacecraft :scream: We need to turn on the thermal vents -- and that means adding more opcodes - specifically 5, 6, 7, and 8

We'll use these to get the code to test the thermal vents

In [15]:
run_diagnostic_tests(intcode, diagnostic_input=5)

8834787